## Joining Fips Codes
- notebook to join fips codes to the redfin data

In [1]:
import pandas as pd
import addfips
import psycopg2

ModuleNotFoundError: No module named 'addfips'

In [5]:
# county level data
county = pd.read_csv(
    "/Users/ianbyrne/Development/redfin_data/county_market_tracker.tsv", sep="\t"
)

In [6]:
county["county"] = county["region"].str.split(",").str[0]
county.head(1)

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated,county
0,2017-04-01,2017-04-30,30,county,5,170,f,"Morgan County, AL",NaN,Alabama,...,NaN,NaN,NaN,0.020833,0.020833,0.001225,"Decatur, AL",19460.0,2022-01-09 14:29:56,Morgan County


In [7]:
row = dict(county.iloc[0])

In [8]:
af = addfips.AddFIPS()
#af.add_county_fips(row, county_field="county", state_field="state_code")

In [9]:
redfin = pd.read_csv("county_fips.csv", dtype={"fips": str})

In [10]:
# null_fips = fips[fips["fips"].isnull()]
# full_fips = fips[fips["fips"].notnull()]

- Baltimore city = 24510
- anchorage borough = 02020
- franklin city = 51620
- fairfax city, va = 51600
- king & queen = 51097
- St louis city = 29510
- richmond city = 51760

In [11]:
stragglers = {
    "Richmond City County, VA": "51760",
    "Anchorage Borough, AK": "02020",
    "Franklin City County, VA": "51620",
    "Fairfax City County, VA": "51600",
    "King & Queen County, VA": "51097",
    "St. Louis City County, MO": "29510",
    "Baltimore City County, MD": "24510",
}

In [12]:
redfin['fips'] = redfin['fips'].fillna(redfin['region'].map(stragglers))
redfin['fips'] = redfin['fips'].astype(int).astype(str)
redfin['fips'] = redfin['fips'].str.zfill(5)

In [13]:
redfin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563122 entries, 0 to 563121
Data columns (total 61 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   fips                            563122 non-null  object 
 1   Unnamed: 1                      563122 non-null  int64  
 2   period_begin                    563122 non-null  object 
 3   period_end                      563122 non-null  object 
 4   period_duration                 563122 non-null  int64  
 5   region_type                     563122 non-null  object 
 6   region_type_id                  563122 non-null  int64  
 7   table_id                        563122 non-null  int64  
 8   is_seasonally_adjusted          563122 non-null  object 
 9   region                          563122 non-null  object 
 10  city                            0 non-null       float64
 11  state                           563122 non-null  object 
 12  state_code      

In [14]:
redfin.head()

,fips,Unnamed: 1,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,...,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated,county
0,01103,0,2017-04-01,2017-04-30,30,county,5,170,f,"Morgan County, AL",...,NaN,NaN,NaN,0.020833,0.020833,0.001225,"Decatur, AL",19460.0,2022-01-09 14:29:56,Morgan County
1,27053,1,2014-12-01,2014-12-31,30,county,5,1457,f,"Hennepin County, MN",...,0.097744,-0.121909,-0.197220,0.172414,0.034483,-0.033469,"Minneapolis, MN",33460.0,2022-01-09 14:29:56,Hennepin County
2,40143,2,2018-02-01,2018-02-28,30,county,5,2319,f,"Tulsa County, OK",...,0.206004,-0.026139,0.009313,0.479924,0.068277,-0.042568,"Tulsa, OK",46140.0,2022-01-09 14:29:56,Tulsa County
3,27035,3,2017-08-01,2017-08-31,30,county,5,1448,f,"Crow Wing County, MN",...,0.200000,-0.050000,NaN,NaN,NaN,NaN,"Brainerd, MN",14660.0,2022-01-09 14:29:56,Crow Wing County
4,28137,4,2019-05-01,2019-05-31,30,county,5,1586,f,"Tate County, MS",...,NaN,NaN,NaN,0.000000,-0.111111,-0.100000,"Memphis, TN",32820.0,2022-01-09 14:29:56,Tate County


In [82]:
redfin.to_csv("county_fips_full.csv")